In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [23]:
import os
from pathlib import Path

import flwr
import torch
from flwr.simulation import run_simulation
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR

from fl_g13.architectures import BaseDino
from fl_g13.fl_pytorch.client_app import get_client_app
from fl_g13.fl_pytorch.server_app import get_server_app

In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
# disable_progress_bar()

Training on cuda
Flower 1.17.0 / PyTorch 2.6.0+cu118


# Login wandb

In [5]:
!pip install wandb


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
## read .env file
import dotenv
dotenv.load_dotenv()


True

In [11]:
import wandb

# login by key in .env file
WANDB_API_KEY = dotenv.dotenv_values()["WANDB_API_KEY"]
wandb.login(key=WANDB_API_KEY)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\ADMIN\_netrc


True

# FL

## Configs

In [24]:
# Model config

## Model Hyper-parameters
head_layers = 3
head_hidden_size = 512
dropout_rate = 0.0
unfreeze_blocks = 1

## Training Hyper-parameters
batch_size = 128
lr = 1e-3
momentum = 0.9
weight_decay = 1e-5
T_max = 8
eta_min = 1e-5

# FL config
K = 100
C = 0.1
J = 4
num_rounds = 20
partition_type = 'iid'

## only for partition_type = 'shard'
num_shards_per_partition = 10

## Server App config
save_every = 1
fraction_fit = C  # Sample of available clients for training
fraction_evaluate = 0.1  # Sample 50% of available clients for evaluation
min_fit_clients = 10  # Never sample less than 10 clients for training
min_evaluate_clients = 5  # Never sample less than 5 clients for evaluation
min_available_clients = 10  # Wait until all 10 clients are available
device = DEVICE
## checkpoints directory
current_path = Path.cwd()
model_test_path = current_path / "../models/fl_baseline"
checkpoint_dir = model_test_path.resolve()
os.makedirs(checkpoint_dir, exist_ok=True)

## Wandb config
use_wandb = True
wandb_config = {
    "fraction_fit": fraction_fit,
    "lr": lr,
    "momentum": momentum,
    'partition_type': partition_type,
    'K': K,
    'C': C,
    'J': J,
}

## simulation run config
NUM_CLIENTS = 100
MAX_PARALLEL_CLIENTS = 10 


## Define model , optimizer and loss function

In [17]:
# Model
model = BaseDino(
    head_layers=head_layers,
    head_hidden_size=head_hidden_size,
    dropout_rate=dropout_rate,
    unfreeze_blocks=unfreeze_blocks
)
model.to(DEVICE)
optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
criterion = torch.nn.CrossEntropyLoss()
scheduler = CosineAnnealingLR(
    optimizer=optimizer,
    T_max=T_max,
    eta_min=eta_min
)

Using cache found in C:\Users\ADMIN/.cache\torch\hub\facebookresearch_dino_main


## Define the ClientApp

## Build module local

Build module local such that ClientApp can use it

In [15]:
!pip install -e ..

Obtaining file:///C:/Users/ADMIN/Desktop/BACKUP/study/Italy/polito/classes/20242/deep%20learning/project/source_code/fl-g13

  Installing build dependencies: started


  Installing build dependencies: finished with status 'done'

  Checking if build backend supports build_editable: started


[notice] A new release of pip is available: 25.0.1 -> 25.1.1

  Checking if build backend supports build_editable: finished with status 'done'

[notice] To update, run: python.exe -m pip install --upgrade pip

  Getting requirements to build editable: started


  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for fl_g13 (pyproject.toml): started
  Building editable for fl_g13 (pyproject.toml): finished with status 'done'
  Created wheel for fl_g13: filename=fl_g13-0.0.1-py3-none-any.whl size=4649 sha256=a588eeccbccdbea56dbfe92e55bd612babd81e07366f39e361ccc99a5ffdec08
  Stored in directory: C:\Users\ADMIN\AppData\Local\Temp\pip-ephem-wheel-cache-zy_t6ks3\wheels\b7\e0\6d\5d22ced2ef400b314cfe74883357cc37e1e1d5275e7ba9175e
Successfully built fl_g13
  Attempting uninstall: fl_g13
    Found existing installation: fl_g13 0.0.1
    Uninstalling fl_g13-0.0.1:
      Successfully uninstalled fl_g13-0.0.1


## Create FlowerClient instances  

### Create instant of ClientApp

In [25]:
client = get_client_app(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    device=DEVICE,
    partition_type=partition_type,
    local_epochs=J,
    batch_size=batch_size,
    num_shards_per_partition=num_shards_per_partition,
    scheduler=scheduler,
    # load_data_fn=load_data_clients
)

# Define the Flower ServerApp

Customize built-in strategy Federated Averaging (FedAvg) of Flower to combine hyperparams in server-side and save model for each k epoch

The strategy could also incremental training 

## Create instant of ServerApp

In [26]:
server = get_server_app(checkpoint_dir=checkpoint_dir,
                        model_class=model,
                        optimizer=optimizer,
                        criterion=criterion,
                        scheduler=scheduler,
                        num_rounds=num_rounds,
                        fraction_fit=fraction_fit,
                        fraction_evaluate=fraction_evaluate,
                        min_fit_clients=min_fit_clients,
                        min_evaluate_clients=min_evaluate_clients,
                        min_available_clients=min_available_clients,
                        device=device,
                        use_wandb=use_wandb,
                        wandb_config=wandb_config,
                        save_every=save_every,
                        prefix='fl_baseline'
                        )

🔍 Loading checkpoint from C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\fl_baseline\FL_BaseDino_epoch_20.pth
📦 Model class in checkpoint: BaseDino
🔧 Model configuration: {'variant': 'dino_vits16', 'dropout_rate': 0.1, 'head_hidden_size': 1024, 'head_layers': 5, 'num_classes': 100, 'unfreeze_blocks': 3, 'activation_fn': 'GELU', 'pretrained': True}


Using cache found in C:\Users\ADMIN/.cache\torch\hub\facebookresearch_dino_main


RuntimeError: Error(s) in loading state_dict for BaseDino:
	Missing key(s) in state_dict: "backbone.cls_token", "backbone.pos_embed", "backbone.patch_embed.proj.weight", "backbone.patch_embed.proj.bias", "backbone.blocks.0.norm1.weight", "backbone.blocks.0.norm1.bias", "backbone.blocks.0.attn.qkv.weight", "backbone.blocks.0.attn.qkv.bias", "backbone.blocks.0.attn.proj.weight", "backbone.blocks.0.attn.proj.bias", "backbone.blocks.0.norm2.weight", "backbone.blocks.0.norm2.bias", "backbone.blocks.0.mlp.fc1.weight", "backbone.blocks.0.mlp.fc1.bias", "backbone.blocks.0.mlp.fc2.weight", "backbone.blocks.0.mlp.fc2.bias", "backbone.blocks.1.norm1.weight", "backbone.blocks.1.norm1.bias", "backbone.blocks.1.attn.qkv.weight", "backbone.blocks.1.attn.qkv.bias", "backbone.blocks.1.attn.proj.weight", "backbone.blocks.1.attn.proj.bias", "backbone.blocks.1.norm2.weight", "backbone.blocks.1.norm2.bias", "backbone.blocks.1.mlp.fc1.weight", "backbone.blocks.1.mlp.fc1.bias", "backbone.blocks.1.mlp.fc2.wei

# Run the training


In [14]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE == "cuda":
    backend_config["client_resources"] = {"num_cpus": 1, "num_gpus": 1}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

### Download missing module for clients

Dino model,that is serialized and sent to client by server, require some modules that have to download from source code of dino model


In [15]:
import os
import urllib.request


def download_if_not_exists(file_path: str, file_url: str):
    """
    Checks if a file exists at the given path. If it does not, downloads it from the specified URL.

    Parameters:
    - file_path (str): The local path to check and save the file.
    - file_url (str): The URL from which to download the file.
    """
    if not os.path.exists(file_path):
        print(f"'{file_path}' not found. Downloading from {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
            print("Download complete.")
        except Exception as e:
            print(f"Failed to download file: {e}")
    else:
        print(f"'{file_path}' already exists.")

In [16]:
download_if_not_exists("vision_transformer.py",
                       "https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/vision_transformer.py")
download_if_not_exists("utils.py",
                       "https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/utils.py")


'vision_transformer.py' already exists.
'utils.py' already exists.


In [18]:
# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config
)

INFO :      Starting Flower ServerApp, config: num_rounds=2, no round_timeout
INFO :      


Continue train model from epoch 2


INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      ROUND 0💡 New best global model found: 0.009400
INFO :      initial parameters (loss, other metrics): 5.01113680948185, {'centralized_accuracy': 0.0094}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)
(ClientAppActor pid=24912) 2025-04-20 22:52:12.922 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13


(ClientAppActor pid=24912) Client 0 is ready to train
(ClientAppActor pid=24912) No prefix/name for the model was provided, choosen prefix/name: bouncy_ivysaur_68
(ClientAppActor pid=24912) 
(ClientAppActor pid=24912) 🚀 Epoch 1/3 (33.33%) Completed
(ClientAppActor pid=24912) 	📊 Training Loss: 5.1139
(ClientAppActor pid=24912) 	✅ Training Accuracy: 1.07%
(ClientAppActor pid=24912) 	⏳ Elapsed Time: 16.03s | ETA: 32.07s
(ClientAppActor pid=24912) 	🕒 Completed At: 22:52
(ClientAppActor pid=24912) 
(ClientAppActor pid=24912) 🚀 Epoch 2/3 (66.67%) Completed
(ClientAppActor pid=24912) 	📊 Training Loss: 5.0673
(ClientAppActor pid=24912) 	✅ Training Accuracy: 1.00%
(ClientAppActor pid=24912) 	⏳ Elapsed Time: 15.22s | ETA: 15.22s
(ClientAppActor pid=24912) 	🕒 Completed At: 22:52
(ClientAppActor pid=24912) 
(ClientAppActor pid=24912) 🚀 Epoch 3/3 (100.00%) Completed
(ClientAppActor pid=24912) 	📊 Training Loss: 4.9448
(ClientAppActor pid=24912) 	✅ Training Accuracy: 0.85%
(ClientAppActor pid=24912) 

INFO :      aggregate_fit: received 10 results and 0 failures


...

INFO :      aggregate_fit: received 10 results and 0 failures


Saving centralized model epoch 3 aggregated_parameters...
💾 Saved checkpoint at: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\fl_baseline\FL_BaseDino_epoch_3.pth


INFO :      ROUND 2💡 New best global model found: 0.018000
INFO :      fit progress: (2, 4.599909758266015, {'centralized_accuracy': 0.018}, 1594.3396579)
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)


(ClientAppActor pid=24912) Client 3 is ready to train
(ClientAppActor pid=24912) Client 4 is ready to train
(ClientAppActor pid=24912) Client 5 is ready to train
(ClientAppActor pid=24912) Client 7 is ready to train
(ClientAppActor pid=24912) Client 8 is ready to train


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 2 round(s) in 1618.09s
INFO :      	History (loss, distributed):
INFO :      		round 1: 5.153822147846222
INFO :      		round 2: 4.96987875699997
INFO :      	History (loss, centralized):
INFO :      		round 0: 5.01113680948185
INFO :      		round 1: 4.689738472805748
INFO :      		round 2: 4.599909758266015
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'federated_evaluate_accuracy': [(1, 0.0086), (2, 0.0116)]}
INFO :      	History (metrics, centralized):
INFO :      	{'centralized_accuracy': [(0, 0.0094), (1, 0.011), (2, 0.018)]}
INFO :      
